# ОПИСАНИЕ
В первой части мы провели анализ данных и построение коллаборативной рекомендательной системы.

Во второй части - content base рекомендательная система основанная на семантическом сходстве контента.

В третьей части мы рассмотрим поиск на основе близости векторов BERT текстового описания контента.

В этой части мы построим рекомендательную систему "холодного сайта"



# ИМПОРТ



In [1]:
import os
import random
import time
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
PATH = '/content/drive/MyDrive/Colab Notebooks/HACKATHONS/Видеопереворот/Рекомендательная система/files/'

# Обезличенная база данных взаимодействия пользователей с видео-контентом (лайки, просмотры, комментарии, загрузки и пр.) 
video_dataset = 'video_dataset.csv'

# Обезличеная база данных видеоконтента 
concurs_data = 'concurs_data.csv'

# ДАННЫЕ

In [4]:
df_concurs = pd.read_csv(PATH + concurs_data)

In [5]:
df_concurs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _key           100000 non-null  int64 
 1   channel_name   99963 non-null   object
 2   comment_count  100000 non-null  int64 
 3   description    95205 non-null   object
 4   dislike_count  100000 non-null  int64 
 5   duration       100000 non-null  int64 
 6   fulltitle      100000 non-null  object
 7   like_count     100000 non-null  int64 
 8   tags           100000 non-null  object
 9   upload_date    100000 non-null  int64 
 10  uploader_url   99998 non-null   object
 11  view_count     100000 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 9.2+ MB


# Решение проблемы холодного старта для нового посетителя
В формате случайной ротации делаем выборку из популярных видео

## Получим список новых видео за указанный период

In [6]:
time_max = df_concurs.sort_values(by='upload_date', ascending=False).iloc[0]['upload_date']
time_min = time_max - 3600 * 24 * 30 * 6  # 6 месяцев
print(f'Даты, min: {time_min}, max: {time_max}')

df_last = df_concurs[df_concurs['upload_date'] > time_min]
print(f'df_last {df_last.shape}')

Даты, min: 1663621200, max: 1679173200
df_last (3894, 12)


## Отсортируем 1000 самых популярных видео

In [7]:
df_popular = df_last.sort_values(by='view_count', ascending=False)[0:1000]

In [8]:
df_popular.head()

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
43440,21658420,ТНТ Смотри еще!,0,Гарик Харламов сделает выбор между любовницей ...,0,718,"Comedy Club: ""Любовница VS Бывшая"" - Гарик Хар...",145879,['тнт онлайн' 'юмор' 'ТНТ' 'шоу тнт' 'камеди к...,1668114000,http://www.youtube.com/channel/UCFKDcjx5ekaoMo...,9451366
76318,67339551,LITVIN,0,Телеграм-канал криптотрейдера Олега Полунина\n...,0,3789,МЕНЯ АРЕСТОВАЛИ ?! УКРАЛИ МАШИНУ В ДУБАЕ ! 2 ч...,1041993,['михаил литвин' 'litvin' 'жизнь' 'life'],1670619600,http://www.youtube.com/@user-bk7hr3pk8s,7142099
63968,29311302,Телеканал ПЯТНИЦА,0,Новые Пацанки — четверг 19:00 на телеканале ПЯ...,0,10884,Новые Пацанки. 8 выпуск. Премьера,83282,['пацанки' 'пацанки новый сезон' 'пацанки 7' '...,1668632400,http://www.youtube.com/user/FridayTVChanel,5390025
50220,67161132,GoB Channel,0,Герой выпуска - Роман Алексеевич Терушкин. \nВ...,0,4135,ЭНДОКРИНОЛОГ / Тяга к Сладкому. Усталость. Сах...,191502,['эндокринолог' 'интервью' 'гоб' 'гоб ченэл' '...,1671829200,http://www.youtube.com/@GoBChannelWorld,5158003
72349,68431820,Comedy Club,0,Смотри Comedy Club по пятницам в 21:00 на ТНТ....,0,641,Камеди Клаб «Неглавные новости 2022» Павел Воля,97878,['камеди клаб' 'камеди' 'тнт' 'тнт онлайн' 'co...,1673298000,http://www.youtube.com/@ComedyClubRussia,4984699


## Получение списка случайного видео

In [9]:
rmd_list = np.random.random_integers(0, 1000, 10)
df_rnd = df_popular[['_key', 'fulltitle']].iloc[rmd_list]
df_rnd.head(10)

<ipython-input-9-5001a52c83f7>:1: DeprecationWarning: This function is deprecated. Please call randint(0, 1000 + 1) instead
  rmd_list = np.random.random_integers(0, 1000, 10)


,_key,fulltitle
34519,71829876,Даша Дива // High-Heels
54137,67354781,Очень странные дела на Юпитере
47444,71321290,Мобилизация в России! Новые поправки УК. Отсро...
29205,67218268,ПРИЕХАЛА ДОЧЬ ПРИВЕЗЛА МАМЕ ГОСТИНЦЫ!
6385,71853520,УСТРОИЛИ БИТВУ НА ЗНАНИЕ ФИЛЬМА! БОЛТАЛЬНЫЙ СП...
76318,67339551,МЕНЯ АРЕСТОВАЛИ ?! УКРАЛИ МАШИНУ В ДУБАЕ ! 2 ч...
6547,72387713,Заслуженная Награда Готова и Конец Боевого Про...
10446,21988154,Как извлечь золото без азотной и серной кислот...
97600,72260063,"Сыч Лучик изгоняет кошку Морошку из совешни, н..."
43518,67342938,ТУПОРЫЛЫЙ ДЕТСКИЙ КОНТЕНТ


# Решение проблемы холодного старта для авторов видео

## Отсортируем 1000 последних видео

Создадим случайную ротацию из последних 1000 видео. Попадание сучайного видео в рекомендательную систему - поможет решить проблему **предвзятости по популярности** и даст шанс попасть в топ новому видео даже не популярного канала. И если поведенческие факторы будут хорошими, то у такого видео будет шанс удержаться в топе.


In [10]:
df_new = df_concurs.sort_values(by='upload_date', ascending=False).iloc[0:1000]
df_new.head(10)

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
19730,72708273,Tilka Play,102,НАШ телеграм канал: https://t.me/tilkaplayTG\n...,0,3168,УЗНАЛА ПРАВДУ о СИСТЕМЕ КОЛЛЕКТИВ ► Atomic Hea...,885,['Atomic Heart' 'атомик харт' 'атомик' 'atomic...,1679173200,http://www.youtube.com/@TilkaPlay,4703
4801,67399612,Майни,92,► Мой МАГАЗИН: https://t.me/MayniShop_bot\n\nЗ...,0,794,За что в Аду могут Посадить в Тюрьму? - «Адски...,2428,[],1679173200,http://www.youtube.com/@MayniYT,15433
70024,67397030,Канал Яла,0,💛 Лучший донат бот https://t.me/yalsquadbot\n💜...,0,983,РАНДОМНЫЕ ТРИО ФОРТНАЙТ БЕЗ СТРОЙКИ,980,['КаналЯла' 'Яла' 'Ял' 'yalexer' 'мемы' 'челле...,1679173200,http://www.youtube.com/@yalexerr,8150
14513,67403483,ZO'R TV,28,Официальный сайт: http://www.zortv.uz\r\nZO'RT...,0,2699,"Yurakdan Gapiramiz 2-mavsum 44-son ""O'Z UYIM B...",0,"[""zo'r tv"" 'zor tv' 'зур тв' 'зор тв']",1679086800,http://www.youtube.com/@ZORTVUZ,6001
44137,72689014,Marvel/DC: Geek Movies,0,Сделай своего питомца супергероем - https://vk...,0,684,Человек-Паук 4 - каким будет сюжет?,937,['Marvel/DC' 'комиксы' 'фильмы' 'Марвел' 'ДС' ...,1679086800,http://www.youtube.com/@MarvelDCRU,7761
69022,68468466,BIG GEEK,0,Полноценно работайте с клиентами и сделками бе...,0,1041,История кнопки Home — как Apple опять изменила...,82,['BIG GEEK' 'биг гик' 'биггик' 'Big Geek' 'гад...,1679086800,http://www.youtube.com/@biggeekru,746
17736,72668668,Телеканал Домашний,0,Новое судебное реалити-шоу на «Домашнем» — это...,0,2935,Давай разведемся | Выпуск 710,35,['давай разведемся' 'домашний' 'телеканал дома...,1679000400,http://www.youtube.com/@DomashniyTV,1482
85619,68446593,Ароматик Bubble,0,🔻🔻🔻🔻🔻🔻🔻🔻🔻🔻🔻🔻🔻🔻\nКиндер Сюрприз👉 Форма мыла htt...,0,987,Дракончик Киндер Сюрприз/2 варианта заливки/Ка...,96,['Мыло' 'Мыловарение' 'Дракон' 'Дракончик' 'Го...,1679000400,http://www.youtube.com/@bubble9121,449
21152,68483887,MELOMAN MUSIC,0,"@MELOMAN-MUSIC \nмузыка, слова - Lonnel, Mauna...",0,145,"Lonnel, Maunavi - Forever or Never (Single 2023)",9,[],1678914000,http://www.youtube.com/@MELOMAN-MUSIC,325
14671,67316758,Stas N,8,Skoda сменила модель А7 на А8. У обоих авто ко...,0,252,Skoda: Гонка А8 DSG VS А7 DSG. Битва Года!!! (...,0,['skoda' 'stas n'],1678914000,http://www.youtube.com/@stasnskodaa8,315


## Получение случайного видео (для единичного пользователя)

In [11]:
rnd = random.randint(0, 1000)
df_rnd = df_new[['_key', 'fulltitle']].iloc[rnd]
df_rnd.head()

_key                                                  28403704
fulltitle    Дети сотрудников ТГК-14 в Чите делают фронтовы...
Name: 2116, dtype: object